In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
import tensorflow as tf
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

### Optimizing Nueral Network using TensorBoard

In [3]:
# dense_layers = [0,1,2]
# layer_sizes = [32, 64, 128]
# conv_layers = [1, 2, 3]

# Optimal network based on Validation Loss in TensorBoard
dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]


### TensorBoard Setup

In [4]:
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0 # Normalize or scale pixel data (max value is 255)

In [7]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
            print(NAME)
            
            model = Sequential()

            # Convulational layer (with 3x3 shift window)
            model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))

            # Additional convulational layers
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten())

            # Additional Dense Layers
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))

            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            model.compile(loss="binary_crossentropy",
                         optimizer="adam",
                          metrics=['accuracy'])

            model.fit(X, y, batch_size=32, epochs=3, validation_split=0.3, callbacks=[tensorboard])
            
model.save('64x3-CNN.model')

3-conv-64-nodes-0-dense-1564278507
Train on 17462 samples, validate on 7484 samples
Epoch 1/3
17462/17462 [==============================] - 46s 3ms/sample - loss: 0.6598 - acc: 0.5963 - val_loss: 0.5959 - val_acc: 0.6871
Epoch 2/3
17462/17462 [==============================] - 45s 3ms/sample - loss: 0.5508 - acc: 0.7236 - val_loss: 0.5262 - val_acc: 0.7410
Epoch 3/3
17462/17462 [==============================] - 46s 3ms/sample - loss: 0.4936 - acc: 0.7647 - val_loss: 0.5311 - val_acc: 0.7277
